In [86]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler

In [87]:
# data preprocess

df = pd.read_csv("BankChurners.csv")
df = df.iloc[:,:-2]

In [88]:
df.set_index('CLIENTNUM', inplace=True)

In [89]:
dfOrigin = df.copy()

In [90]:
df = pd.get_dummies(df)

In [91]:
scaler = MinMaxScaler()
dfNorm = scaler.fit_transform(df)

In [93]:
# Kmeans Clustering
km = MiniBatchKMeans(n_clusters=4)
dfPred = km.fit_predict(dfNorm)

In [94]:
df['grp'] = dfPred
dfOrigin['grp'] = dfPred

In [97]:
df.groupby('grp').mean().iloc[:,10:20]

,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag_Attrited Customer,Attrition_Flag_Existing Customer,Gender_F,Gender_M,Education_Level_College,Education_Level_Doctorate
grp,,,,,,,,,,
0,4469.087529,68.521882,0.709856,0.332319,0.179294,0.820706,1.0,0.0,0.097412,0.045647
1,4229.163625,64.926384,0.715937,0.348291,0.169811,0.830189,1.0,0.0,0.100526,0.049490
2,4244.438283,59.597048,0.714976,0.219116,0.132826,0.867174,0.0,1.0,0.101968,0.035331
3,4713.746940,66.343861,0.707036,0.182274,0.157916,0.842084,0.0,1.0,0.099882,0.045401


In [65]:
df.columns

Index(['Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Gender_F', 'Gender_M', 'Education_Level_College',
       'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Education_Level_Unknown',
       'Marital_Status_Divorced', 'Marital_Status_Married',
       'Marital_Status_Single', 'Marital_Status_Unknown',
       'Income_Category_$120K +', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Income_Category_Unknown',
       'Card_Category_Blue', 'Card_Category_Gold', 'Card_Category_Platinum',
       'Car